# Alzheimer's Disease Drugs Repurposing via pre-trained embedding

This is an example showing that conduct AD drug repurposing by using iBKH with pre-trained embedding information. We used pre-trained iBKH embedding information from different models as input, then predicted the most likely associated entity with Alzheimer's Disease (AD).

In [ ]:
import pandas as pd
import numpy as np

import torch as th
import torch.nn.functional as fn

from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt

### Predict Score Functions

We used the following algorithms to calculate the edge scores. And the edge scores indicate the strength of association between candidate entities and AD.

In [ ]:
def transE_l2(head, rel, tail, gamma=12.0):
    score = head + rel - tail
    return gamma - th.norm(score, p=2, dim=-1)

In [ ]:
def transR(head, rel, tail, proj, rel_idx, gamma=12.0):
    proj = proj.reshape(-1, head.shape[1], rel.shape[0])[rel_idx]
    head_r = th.einsum('ab,bc->ac', head, proj)
    tail_r = th.einsum('b,bc->c', th.tensor(tail), proj)
    score = head_r + rel - tail_r
    return gamma - th.norm(score, p=1, dim=-1)

In [ ]:
def DistMult(head, rel, tail):
    score = head * rel * tail
    return th.sum(score, dim=-1)

In [ ]:
def complEx(head, rel, tail, gamma=12.0):
    real_head, img_head = th.chunk(head, 2, dim=-1)
    real_tail, img_tail = th.chunk(th.tensor(tail), 2, dim=-1)
    real_rel, img_rel = th.chunk(rel, 2, dim=-1)

    score = real_head * real_tail * real_rel \
            + img_head * img_tail * real_rel \
            + real_head * img_tail * img_rel \
            - img_head * real_tail * img_rel
    # TODO: check if there exists minus sign and if gamma should be used here(jin)
    return th.sum(score, -1)

### XModel Prediction

We used pre-trained embedding information from the different models to predict the association between the candidate drugs and AD. All the candidate drugs are FDA-approved. We rank the results based on the predict score functions. The larger the score is, the stronger association between candidate drugs and AD.

In [ ]:
folder = 'Data/'
candidate_D_folder = 'Data/candidate_drugs/'
kg_folder = '../iBKH/iBKH_May_3/'
result_folder = 'predict_result/'

In [ ]:
def xModel_prediction(model_name, trail_status):
    entity_df = pd.read_table(folder + model_name + '/entities.tsv', header=None)
    entity_df = entity_df.dropna().reset_index(drop=True)
    approved_drug_df = pd.read_csv(candidate_D_folder + 'candidate_drugs_' + trail_status + '.csv')
    approved_drug_list = list(approved_drug_df['Drug'])

    entity_map = {}
    entity_id_map = {}
    relation_map = {}
    drug_ids = []
    drug_names = []
    disease_ids = []

    for i in range(len(entity_df)):
        entity_id = entity_df.loc[i, 0]
        entity_name = entity_df.loc[i, 1]
        entity_map[entity_name] = int(entity_id)
        entity_id_map[int(entity_id)] = entity_name
        if entity_name.replace('DrugBank:', '') in approved_drug_list:
            drug_ids.append(entity_id)
            drug_names.append(entity_name.replace('DrugBank:', ''))

    disease_vocab = pd.read_csv(kg_folder + 'Entity/disease_vocab.csv')
    AD_related_list = []
    for i in range(len(disease_vocab)):
        primary_id = disease_vocab.loc[i, 'primary']
        disease_name = disease_vocab.loc[i, 'name']
        disease_name = disease_name if not pd.isnull(disease_name) else ''
        if 'alzheimer' in disease_name:
            if primary_id not in AD_related_list:
                AD_related_list.append(primary_id)

    relation_df = pd.read_table(folder + model_name + '/relations.tsv', header=None)
    for i in range(len(relation_df)):
        relation_id = relation_df.loc[i, 0]
        relation_name = relation_df.loc[i, 1]
        relation_map[relation_name] = int(relation_id)

    for disease in AD_related_list:
        if disease in entity_map:
            disease_ids.append(entity_map[disease])

    entity_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_entity.npy')
    rel_emb = np.load(folder + model_name + '/iBKH_' + model_name + '_relation.npy')
    if model_name == 'TransR':
        proj_np = np.load(folder + 'TransR/iBKH_TransRprojection.npy')
        proj_emb = th.tensor(proj_np)

    treatment = ['Treats_DDi', 'Palliates_DDi', 'Effect_DDi', 'Associate_DDi', 'Inferred_Relation_DDi',
                 'Semantic_Relation_DDi']
    treatment_rid = [relation_map[treat] for treat in treatment]

    drug_ids = th.tensor(drug_ids).long()
    disease_ids = th.tensor(disease_ids).long()
    treatment_rid = th.tensor(treatment_rid)

    drug_emb = th.tensor(entity_emb[drug_ids])
    treatment_embs = [th.tensor(rel_emb[rid]) for rid in treatment_rid]

    scores_per_disease = []
    dids = []
    for rid in range(len(treatment_embs)):
        treatment_emb = treatment_embs[rid]
        for disease_id in disease_ids:
            disease_emb = th.tensor(entity_emb[disease_id])
            if model_name == 'RotatE':
                score = fn.logsigmoid(rotatE(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'ComplEx':
                score = fn.logsigmoid(complEx(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'TransR':
                score = fn.logsigmoid(transR(drug_emb, treatment_emb, disease_emb, proj_emb, treatment_rid[rid]))
            elif model_name == 'TransE_l2':
                score = fn.logsigmoid(transE_l2(drug_emb, treatment_emb, disease_emb))
            elif model_name == 'DistMult':
                score = fn.logsigmoid(DistMult(drug_emb, treatment_emb, disease_emb))
            scores_per_disease.append(score)
            dids.append(drug_ids)
    scores = th.cat(scores_per_disease)
    dids = th.cat(dids)

    idx = th.flip(th.argsort(scores), dims=[0])
    scores = scores[idx].numpy()
    dids = dids[idx].numpy()

    _, unique_indices = np.unique(dids, return_index=True)
    topk_indices = np.sort(unique_indices)
    proposed_dids = dids[topk_indices]
    proposed_scores = scores[topk_indices]

    candidate_drug_rank = []
    candidate_drug_score = {}
    for i, idx in enumerate(proposed_dids):
        candidate_drug_rank.append(entity_id_map[int(idx)].replace('DrugBank:', ''))
        candidate_drug_score[entity_id_map[int(idx)].replace('DrugBank:', '')] = proposed_scores[i]

    df = pd.DataFrame(columns=['Drug', 'Score'])
    idx = 0
    for drug in candidate_drug_score:
        df.loc[idx] = [drug, candidate_drug_score[drug]]
        idx += 1

    x = np.asarray(df['Score']).reshape(-1, 1)  # returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df['Score_scaled'] = pd.DataFrame(x_scaled)
    print(df)
    df.to_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trail_status + ".csv", index=False)

### ROC Curve

We have identified different sets of ground truths based on the clinical trails status of candidate drugs. We used the ranked lists (predict results) obtained from the embedding information to calculate ROC and AUC scores to analyze the performance. The ROC was plotted based on the different thresholds of the false positive and true positive rates. The AUC scores range from 0 to 1, where 1 corresponds to perfect performance and 0.5 indicates random classical performance.

In [ ]:
def generate_AUC(model_name):
    figures_folder = result_folder + 'auc_figures/'
    drug_trail_list = ['approve_phase1234', 'approve_phase234', 'approve_phase34', 'approve_phase4', 'approve']
    trail_info = {'approve_phase1234': {'label': 'FDA approved+Phase I~IV', 'color': '#6a4c93'},
                  'approve_phase234': {'label': 'FDA approved+Phase II~IV', 'color': '#1982c4'},
                  'approve_phase34': {'label': 'FDA approved+Phase III,IV', 'color': '#8ac926'},
                  'approve_phase4': {'label': 'FDA approved+Phase IV', 'color': '#ffca3a'},
                  'approve': {'label': 'FDA approved', 'color': '#ff595e'}}
    plt.figure(figsize=(7, 7))

    for trail_status in drug_trail_list:
        predict_res = pd.read_csv(result_folder + "predict_result_scaled_" + model_name + "_" + trail_status + ".csv")
        candidate_df = pd.read_csv(candidate_D_folder + 'candidate_drugs_' + trail_status + '.csv')

        df = pd.merge(predict_res, candidate_df, on='Drug')

        label = np.array(list(df['label']))
        score = np.array(list(df['Score_scaled']))
        # score = np.array(list(df['Score']))

        fpr, tpr, thresholds = metrics.roc_curve(label, score)
        youden = tpr - fpr
        youden_J = np.max(youden)
        inds_youden_J = np.where(youden == youden_J)
        tpr_max = tpr[inds_youden_J]
        fpr_max = fpr[inds_youden_J]
        cut_off = thresholds[inds_youden_J][0]
        sensitivity = tpr_max[0]
        specificity = 1 - fpr_max[0]
        prevalence = np.where(label == 1)[0].shape[0] / label.shape[0]
        acc = (sensitivity * prevalence) + (specificity * (1 - prevalence))
        print(cut_off, sensitivity, specificity, acc)
        auc = metrics.auc(fpr, tpr)
        plt.plot(fpr, tpr, label=trail_info[trail_status]['label'] + ', AUC=' + str(round(auc, 2)),
                 color=trail_info[trail_status]['color'])

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='gray', alpha=.8)
    plt.tick_params(labelsize=16, bottom=True, left=True)
    plt.xlabel("1 - Specificity", fontsize=12, fontweight='bold')
    plt.ylabel("Sensitivity", fontsize=12, fontweight='bold')
    plt.grid(alpha=.3)
    plt.legend(prop={'size': 12}, loc=4)
    plt.title(model_name, fontweight='bold', fontsize=18)
    # plt.show()
    plt.savefig(figures_folder + model_name + '.pdf', dpi=300)
    plt.close()

In [ ]:
drug_trail_list = ['approve_phase1234', 'approve_phase234', 'approve_phase34', 'approve_phase4', 'approve']

### Prediction & ROC curve (TransE)

In [ ]:
model_name = 'TransE_l2'
for trail_status in drug_trail_list:
    xModel_prediction(model_name, trail_status)

generate_AUC(model_name)

### Prediction & ROC curve (TransR)

In [ ]:
model_name = 'TransR'
for trail_status in drug_trail_list:
    xModel_prediction(model_name, trail_status)

generate_AUC(model_name)

### Prediction & ROC curve (DistMult)

In [ ]:
model_name = 'DistMult'
for trail_status in drug_trail_list:
    xModel_prediction(model_name, trail_status)

generate_AUC(model_name)

### Prediction & ROC curve (ComplEx)

In [ ]:
model_name = 'ComplEx'
for trail_status in drug_trail_list:
    xModel_prediction(model_name, trail_status)

generate_AUC(model_name)